## Notebook 4 - Analysis of MS spectra

This notebook identifies products based on the database of MS2 spectra generated previously.

In [ ]:
%run ../common.py

In [2]:
from pyopenms import MSExperiment, MzMLFile

Determination of memory status is not supported on this 
 platform, measuring for memoryleaks will never fail


In [ ]:
# We already prepared the reference database to analyze the main screen
df_database = pd.read_pickle('../screening_notebooks/results/MS2_database_shifts_162_320_VB_clean.pkl')

In [4]:
def check_c_glucoside(mz_peaks, i_peaks, charge, mz_precursor, adduct_type, tol=0.05, i_threshold=0.10):
    shifts = np.array([120.042, 90.032])
    if charge == 0:
        if adduct_type in ['M+Glu+H','M+Glu+Na','M+Glu+NH4','M+Glu+ACN+H','M+2Glu+H','M+2Glu+Na','M+2Glu+NH4','M+2Glu+ACN+H']:
            pass
        elif adduct_type in ['M+Glu+2H','M+2Glu+2H']:
            shifts = shifts / 2.
    elif charge ==1:
        if adduct_type in ['M+Glu+H', 'M+2Glu+H']:
            pass
        elif adduct_type in ['M+Glu+2H','M+Glu+Na','M+Glu+NH4','M+Glu+ACN+H','M+2Glu+2H','M+2Glu+Na','M+2Glu+NH4','M+2Glu+ACN+H']:
            shifts = shifts / 2.
            
    i_peaks = i_peaks / max(i_peaks)
    
    for shift in shifts:
        test = np.where(np.abs(mz_precursor - mz_peaks - shift) < tol)[0]
        for i in test:
            if i_peaks[i] > i_threshold:
                return True
    return False

In [5]:
def integrate_MS1_row(row, exp, verbose):

    if verbose:
        print(f"Product {row['Name']} identified with score {row['CosineScore']}")

    rt_hit = row['RT']
    mz_hit = row['PrecursorMZ']

    AUC = []
    t_integrate = []
    mz_ub = mz_hit*(1+ppm_integrate/1e6)
    mz_lb = mz_hit*(1-ppm_integrate/1e6)
    rt_lb = rt_hit - rt_semiwindow_integrate
    rt_ub = rt_hit + rt_semiwindow_integrate
    for spec in exp:
        if spec.getMSLevel() == 1:
            rt = spec.getRT()
            if rt < rt_lb:
                continue
            if rt > rt_ub:
                break


            # Let us integrate along the m/z dimension to obtain the AUC for that time slice
            mz_peaks, i_peaks = spec.get_peaks()
            idx_bool = (mz_peaks > mz_lb) & (mz_peaks < mz_ub)
            if any(idx_bool):
                i_peaks_integrate = np.concatenate(([0.], i_peaks[idx_bool], [0.]))
                mz_peaks_integrate = np.concatenate(([mz_lb], mz_peaks[idx_bool], [mz_ub]))
                AUC_i = np.trapz(y=i_peaks_integrate, x=mz_peaks_integrate)

                # Let us compute the baseline AUC to be substracted
                #idx_where = np.where(idx_bool)[0]

                #idx_first = idx_where[0]
                #idx_last = idx_where[-1]
                #i_peaks_integrate = np.array([0., i_peaks[idx_first], i_peaks[idx_last], 0.])
                #mz_peaks_integrate = np.array([mz_lb, mz_peaks[idx_first], mz_peaks[idx_last], mz_ub])
                #AUC_base = np.trapz(y=i_peaks_integrate, x=mz_peaks_integrate)
                AUC_base = 0.

                t_integrate.append(rt)
                AUC.append(AUC_i-AUC_base)

    # Let us integrate along the time dimension
    t_integrate = np.concatenate(([rt_lb], t_integrate, [rt_ub]))
    AUC = np.concatenate(([0.], AUC, [0.]))
    AUC = np.trapz(y=AUC, x=t_integrate)
        
    return AUC
        

What we will do is to record all hits with `CosineScore` > 0.5. Then, we can plot the number of hits vs. the desired cutoff (say CosineScore = 0.85).

In [6]:
score_threshold = 0.5

In [ ]:
def process_mix(rt_dict={}, verbose=False):
    
    ### Prepare reference
    df_mix = df_database #[df_database['Mix'] == mix_no]

    ref_name = list(df_mix['Name'])
    ref_smiles = list(df_mix['CSMILES'])
    ref_substrates_charges = np.array(df_mix['M_charge'])
    colnames = [a for a in df_mix.columns if 'M+' in a]
    ref_precursors_mz = np.array(df_mix[colnames])
    ref_fragments_mz = list(df_mix['mz'])
    ref_fragments_i = list(df_mix['relint'])
    ref_mona_no = list(df_mix['DB#'])
    
    ###
    
    files = glob.glob(f'../data/validation_data/mzML/*.mzML')
    
    df = pd.DataFrame(columns = ['File', 'Enzyme_id', 'Name', 'Rep', 'CSMILES', 'PrecursorMZ', 'Adduct', 'RT', 
                                 'CosineScore', 'C-gly', 'AUC'])
    c = 0 
    for filename in files:
        c+=1
        file = os.path.basename(filename)
        print(f'\n{file} (file {c} out of {len(files)})')

        enzyme_id = file.split('_')[0]
        
        exp = MSExperiment()
        MzMLFile().load(filename, exp)


        ## SUBSET EXPERIMENTAL MS2 SPECTRA

        list_hit_name = []
        list_hit_mzprecursor = []
        list_hit_rt = []
        list_hit_score = []
        list_hit_smiles = []
        list_adducts = []
        list_c_glu = []
        list_mona_no = []
        for spec in exp:
            if spec.getMSLevel() == 2:
                mz_precursor = spec.getPrecursors()[0].getMZ()
                mz_peaks, i_peaks = spec.get_peaks()
                rt = spec.getRT() # minutes
                
                test1 = ((np.abs(mz_precursor - ref_precursors_mz)/mz_precursor)*1e6) < ppm # boolean 2D matrix
                
                score_best = score_threshold
                for i,j in zip(*np.where(test1)):
                    
                    # We also test for retention time
                    name = ref_name[i]
                    ref_rt = rt_dict.get(name, None)
                    if ref_rt is None: # If not have a reference RT for a compound, assume any value is possible
                        test2 = True
                    else:
                        test2 = np.abs(rt - ref_rt) < rt_semiwindow_cutoff # boolean scalar
                    
                    if test2:
                        score = cosine_greedy_score(mz_peaks, i_peaks, ref_fragments_mz[i], ref_fragments_i[i])
                        if score > score_best:
                            score_best = score
                            I = i
                            J = j
                
                if score_best > score_threshold:
                    list_hit_mzprecursor.append(mz_precursor)
                    
                    list_hit_rt.append(rt)
                    list_hit_score.append(score_best)
                    list_hit_smiles.append(ref_smiles[I])
                    list_mona_no.append(ref_mona_no[I])
                    adduct_type = colnames[J]
                    list_adducts.append(adduct_type)
                    name = ref_name[I]
                    if '2Glu' in adduct_type:
                        name += ' (2GLC)'
                    list_hit_name.append(name)
                    
                    # We consider that the hit may be a C-glycoside
                    # if it has a -120 or -90 Da neutral loss
                    charge = ref_substrates_charges[I]
                    is_c_glu = check_c_glucoside(mz_peaks, i_peaks, charge, mz_precursor, adduct_type)
                    list_c_glu.append(is_c_glu)
                    
        
        df_temp = pd.DataFrame({
            'File': filename,
            'Substrate_used': file.split('_')[1],
            'Rep': file.split('_')[2][0],
            'Enzyme_id': enzyme_id,
            'Name': list_hit_name,
            'CSMILES': list_hit_smiles,
            'PrecursorMZ': np.round(list_hit_mzprecursor,4),
            'Adduct': list_adducts,
            'RT': np.round(list_hit_rt, 2),
            'CosineScore': np.round(list_hit_score,3),
            'C-gly': list_c_glu,
            'MoNA_DB#': list_mona_no,
        })

        # Let us now remove duplicate products based on their name
        # The sorting ensures we keep the highest score when dropping
        df_temp.sort_values(by=['CosineScore'], ascending=[False], inplace=True) 
        df_temp = df_temp.drop_duplicates(subset=['Name'], keep='first') # We will keep the highest score for each Name
        
        print(f'{len(df_temp)} products identified')
        

        ## INTEGRATE IN THE MS1 CHANNEL (2D integration)
        
        list_hit_AUC = df_temp.apply(integrate_MS1_row, args=(exp, False), axis=1)

        df_temp['AUC'] = np.round(list_hit_AUC,0)
        df = pd.concat([df,df_temp], ignore_index=True)
    
    return df

In [ ]:
%%time

rt_dict = {}
i = 1
while True:
    df = pd.DataFrame()
    print(f'\nIteration {i}')

    df_tmp = process_mix(rt_dict)
    if not df_tmp.empty:
        df = pd.concat([df,df_tmp])
        
    df.to_pickle(f'./tmp/Val_results_cosine_iteration_{i}.pkl')
    df.to_csv(f'./tmp/Val_results_cosine_iteration_{i}.csv', index=False)
    
    # Let us update the rt_dict for the next iteration
    # To compute the dictionary, we get the median RT for each substrate observed across all experiments
    # I will only use reliable hits for computing the dictionary (i.e., CosineScore>=0.85)
    df_tmp = df[df['CosineScore']>=0.85]
    rt_dict_new = dict(df_tmp.groupby(['Name'])['RT'].median())
    
    # Let us only run 1 iteration
    if (rt_dict_new == rt_dict) | i == 1:
        break
        
    rt_dict = rt_dict_new
    i+=1


Iteration 1

73C5_GlycocholicAcid_1.mzML (file 1 out of 96)
8 products identified

73C5_Osajin_3.mzML (file 2 out of 96)


/var/folders/v1/xrsbvyx519g764hlvcqf01p40000gq/T/ipykernel_6102/3598349742.py:118: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,df_temp], ignore_index=True)


10 products identified

73C5_Lobeline_3.mzML (file 3 out of 96)
9 products identified

73C5_Steviol_3.mzML (file 4 out of 96)
9 products identified

73C5_Boldine_3.mzML (file 5 out of 96)
6 products identified

73C5_Capsaicin_1.mzML (file 6 out of 96)
8 products identified

73C5_ChlorogenicAcid_2.mzML (file 7 out of 96)
7 products identified

73C5_AleureticAcid_2.mzML (file 8 out of 96)
8 products identified

73C5_246trihydroxyacetophenone_3.mzML (file 9 out of 96)
8 products identified

73C5_Curcumin_2.mzML (file 10 out of 96)
9 products identified

73C5_Cortisone_2.mzML (file 11 out of 96)
8 products identified

73C5_Esculetin_3.mzML (file 12 out of 96)
7 products identified

73C5_Ononetin_2.mzML (file 13 out of 96)
7 products identified

73C5_Colforsin_1.mzML (file 14 out of 96)
8 products identified

73C5_Formononetin_3.mzML (file 15 out of 96)
6 products identified

73C5_Formononetin_2.mzML (file 16 out of 96)
7 products identified

73C5_Ononetin_3.mzML (file 17 out of 96)
10 prod